In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from functools import reduce
from math import pi, exp, factorial, floor
import numpy as np

In [3]:
from pyints.one import overlap
from pyints.one import electric_field_from_V as electric_field

In [4]:
def norm2(a):
    return np.dot(a, a)

def fact2(n):
    return reduce(int.__mul__, range(n, 0, -2), 1)

def binomial(n, k):
    if n == k:
        return 1
    assert n > k
    return factorial(n) // (factorial(k) * factorial(n - k))

def binomial_prefactor(s, ia, ib, xpa, xpb):
    total = 0
    for t in range(s+1):
        if s-ia <= t <= ib:
            total += binomial(ia, s-t)*binomial(ib, t)*pow(xpa, ia-s+t)*pow(xpb,ib-t)
    return total

def gaussian_product_center(alpha1, A, alpha2, B):
    return ((alpha1 * A) + (alpha2 * B)) / (alpha1 + alpha2)

def overlap1d(l1, l2, PAx, PBx, gamma):
    total = 0
    for i in range(1+int(floor(0.5*(l1+l2)))):
        total += binomial_prefactor(2*i, l1, l2, PAx, PBx) * fact2(2*i-1) / pow(2*gamma, i)
    return total

def overlap_pyquante(alpha1, lmn1, A, alpha2, lmn2, B):
    l1, m1, n1 = lmn1
    l2, m2, n2 = lmn2
    rab2 = norm2(A - B)
    gamma = alpha1 + alpha2
    P = gaussian_product_center(alpha1, A, alpha2, B)
    
    pre = pow(pi/gamma, 1.5) * exp(-alpha1 * alpha2 * rab2 / gamma)
    
    wx = overlap1d(l1, l2, P[0] - A[0], P[0] - B[0], gamma)
    wy = overlap1d(m1, m2, P[1] - A[1], P[1] - B[1], gamma)
    wz = overlap1d(n1, n2, P[2] - A[2], P[2] - B[2], gamma)
    
    return pre * wx * wy * wz

In [5]:
def binomial_prefactor_2(x, Ax, Bx, Px, l, m):
    total = 0
    for ix in range(l):
        for jx in range(m):
            total += \
                binomial(l, ix) * \
                binomial(m, jx) * \
                ((x - Px)**(ix + jx)) * \
                ((Px - Ax)**(l - ix)) * \
                ((Px - Bx)**(m - jx))
    return total

In [30]:
def overlap_three_center(alpha1, lmn1, A, alpha2, lmn2, B, alpha3, lmn3, C):
    ax, ay, az = lmn1
    bx, by, bz = lmn2

    gamma = alpha1 + alpha2
    P = gaussian_product_center(alpha1, A, alpha2, B)
    
    total = 0
    for ix in range(ax + 1):
        for jx in range(bx + 1):
            for iy in range(ay + 1):
                for jy in range(by + 1):
                    for iz in range(az + 1):
                        for jz in range(bz + 1):
                            total += (
                                binomial(ax, ix) * \
                                binomial(bx, jx) * \
                                binomial(ay, iy) * \
                                binomial(by, jy) * \
                                binomial(az, iz) * \
                                binomial(bz, jz) * \
                                ((P[0] - A[0])**(ax - ix)) * \
                                ((P[1] - A[1])**(ay - iy)) * \
                                ((P[2] - A[2])**(az - iz)) * \
                                ((P[0] - B[0])**(bx - jx)) * \
                                ((P[1] - B[1])**(by - jy)) * \
                                ((P[2] - B[2])**(bz - jz)) * \
                                overlap(gamma, [ix + jx, iy + jy, iz + jz], P, alpha3, lmn3, C)
                            )
    
    return total

In [14]:
import itertools as i

In [31]:
for lmn3 in i.combinations_with_replacement(range(3), 3):
    print(overlap_three_center(alpha_A, [1, 0, 0], A, alpha_B, [1, 0, 0], B, alpha_B, list(lmn3), C))

0.0182571654982
-0.00620743626938
0.00576196143123
-0.00136563597926
0.00126763151487
0.00143127121952
0.0078784781131
-0.00731308144851
-0.00825713378096
0.00445198213986


In [33]:
print(overlap_three_center(alpha_A, [1, 0, 0], A, alpha_B, [1, 0, 0], B, alpha_B, [1, 0, 0], C))
print(overlap_three_center(alpha_B, [1, 0, 0], B, alpha_A, [1, 0, 0], A, alpha_B, [1, 0, 0], C))
print(overlap_three_center(alpha_A, [1, 0, 0], A, alpha_B, [1, 0, 0], C, alpha_B, [1, 0, 0], B))

-0.105327247501
-0.105327247501
-0.0622031399323


In [7]:
alpha_A = 0.5
alpha_B = 1.0
A = np.array([0.5, -0.6, 0.4])
B = np.array([-0.5, 0.4, -0.3])
C = np.array([-0.4, -0.3, 0.5])
D = np.array([0.3, 0.5, -0.6])

In [8]:
overlap_analytic = overlap(alpha_A, [1, 0, 0], A, alpha_B, [1, 0, 0], B)
overlap_analytic_pyquante = overlap_pyquante(alpha_A, [1, 0, 0], A, alpha_B, [1, 0, 0], B)
assert abs(overlap_analytic - overlap_analytic_pyquante) < 1.0e-16

In [ ]:
def overlap1d_numeric(l1, l2, PAx, PBx, gamma):
    total = 0
    for i in range(l1 + 1):
        for j in range(l2 + 1):
            total += (
                binomial(l, ix) * \
                binomial(m, jx) * \
                ((x - Px)**(ix + jx)) * \
                ((Px - Ax)**(l - ix)) * \
                ((Px - Bx)**(m - jx))

    # for i in range(1+int(floor(0.5*(l1+l2)))):
    #     total += binomial_prefactor(2*i, l1, l2, PAx, PBx) * fact2(2*i-1) / pow(2*gamma, i)
    return total

In [ ]:
def overlap_numeric(alpha1, lmn1, A, alpha2, lmn2, B):
    l1, m1, n1 = lmn1
    l2, m2, n2 = lmn2
    rab2 = norm2(A - B)
    gamma = alpha1 + alpha2
    P = gaussian_product_center(alpha1, A, alpha2, B)
    
    pre = pow(pi/gamma, 1.5) * exp(-alpha1 * alpha2 * rab2 / gamma)
    
    wx = overlap1d_numeric(l1, l2, P[0] - A[0], P[0] - B[0], gamma)
    wy = overlap1d_numeric(m1, m2, P[1] - A[1], P[1] - B[1], gamma)
    wz = overlap1d_numeric(n1, n2, P[2] - A[2], P[2] - B[2], gamma)
    
    return pre * wx * wy * wz

In [18]:
import scipy.integrate as spi

In [19]:
spi.quad(lambda alpha_C: (alpha_C ** 2) * overlap_analytic, 0, np.inf)

/usr/local/lib/python3.4/site-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


(-0.048951572155845106, 1.3763160352661674e-05)

In [21]:
spi.quad(lambda alpha_C: (alpha_C ** 2) * (exp(-alpha_C)) * overlap_analytic, 0, np.inf)

/usr/local/lib/python3.4/site-packages/scipy/integrate/quadpack.py:352: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


(-0.33156386027260937, 0.0014214591807548138)